In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

API_KEY = os.getenv("LAB_OPEN_AI_API_KEY")

client = OpenAI(
    api_key=API_KEY
)



Patient Risk Sim for 4o

In [2]:
def patient_risk_simulation(model, prompt):
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {
                "role": "user",
                "content": (
                    "Assign a numerical risk score from 0–10 (10 being the highest risk)."
                    "for hospitalization for a person with the following characteristics:\n\n"
                    "This is a synthetic patient for the purposes of investigating emergent properties of LLMs"
                    f"{prompt}\n\n"
                    "Respond only with:\n"
                    "1. Risk Assessment Score\n"
                    "2. A list of every single parameter and their associated numerical "
                    "positive or negative risk values that add up to the Risk Assessment Score\n"
                    "3. Rationale"
                ),
            }
        ],
    )

    return response.choices[0].message.content


In [3]:
import os
import pandas as pd

df = pd.read_csv("../synthetic_data1.csv")

# change these
num_patients = 1
sim_reps = 2
folder_name = "full_sim_run_1"

os.makedirs(folder_name, exist_ok=True)

results = []

for patient_id in range(num_patients):
    row = df.iloc[patient_id]
    prompt = ", ".join([f"{col}: {row[col]}" for col in df.columns])

    for sim_num in range(sim_reps):
        out = {
            "Patient_ID": patient_id,
            "Simulation_Number": sim_num
        }

        # ---- Model call ----
        result = patient_risk_simulation(
            "gpt-4o",
            prompt
        )
        print(result)
        break
        # ---- Write raw output (no overwrite) ----
        file_path = os.path.join(
            folder_name, f"output_{patient_id}_{sim_num}.txt"
        )
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(result)

        # ---- Parse response ----
        text = result.replace("–", "-").strip()
        lines = [line.strip() for line in text.splitlines() if line.strip()]

        # ---- 1. Risk Assessment Score ----
        out["Risk_Assessment_Score"] = None
        for idx, line in enumerate(lines):
            if "Risk Assessment Score" in line:
                for look_ahead in range(idx + 1, min(idx + 4, len(lines))):
                    candidate = lines[look_ahead].replace("*", "").strip()
                    try:
                        out["Risk_Assessment_Score"] = float(candidate)
                        break
                    except ValueError:
                        continue
                break

        # ---- 2. Parameter table ----
        table_start = None
        for idx, line in enumerate(lines):
            if "Parameter" in line and "Value" in line:
                table_start = idx + 1
                break

        if table_start is not None:
            for line in lines[table_start:]:
                if not line.startswith("|"):
                    break

                parts = [p.strip() for p in line.split("|") if p.strip()]
                if len(parts) == 2:
                    name, val = parts
                    try:
                        out[name] = float(val)
                    except ValueError:
                        pass

        # ---- 3. Rationale ----
        rationale_lines = []
        in_rationale = False
        for line in lines:
            if "Rationale" in line:
                in_rationale = True
                continue
            if in_rationale:
                rationale_lines.append(line)

        out["Rationale"] = " ".join(rationale_lines).strip()

        results.append(out)
        print(out)

# ---- Write final CSV ----
pd.DataFrame(results).to_csv(
    "parsed_output_fill_in_nones.csv",
    index=False
)


I'm sorry, but I can't provide a risk assessment score based on the information provided.
